# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), December 12, 2021**

In [1]:
using DataFrames

In [2]:
using BenchmarkTools

In [3]:
using CategoricalArrays

In [4]:
using PooledArrays

## Performance tips

### Access by column number is faster than by name

In [5]:
x = DataFrame(rand(5, 1000), :auto)
@btime $x[!, 500];
@btime $x.x500;

  4.905 ns (0 allocations: 0 bytes)
  30.404 ns (1 allocation: 32 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [6]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  175.182 ms (5999017 allocations: 122.06 MiB)


In [7]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  9.406 ms (37 allocations: 30.52 MiB)
  9.612 ms (37 allocations: 30.52 MiB)


In [8]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2), :auto)
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  8.894 ms (37 allocations: 30.52 MiB)


In general for tall and narrow tables it is often useful to use `Tables.rowtable`, `Tables.columntable` or `Tables.namedtupleiterator` for intermediate processing of data in a type-stable way.

### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between copying vs non-copying data frame creation

In [9]:
function f1()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto, copycols=false) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100], :auto) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto, copycols=false) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x, :auto) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  36.928 ms (1949523 allocations: 37.40 MiB)
  28.738 ms (1949723 allocations: 45.03 MiB)
  2.267 ms (623 allocations: 7.66 MiB)
  3.349 ms (823 allocations: 15.30 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [10]:
x = DataFrame(rand(10^6, 5), :auto)
y = DataFrame(transpose(1.0:5.0), :auto)
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5), :auto) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  7.293 ms (180 allocations: 38.15 MiB)
  1.040 μs (17 allocations: 672 bytes)
  588.525 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [11]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.031 ms (7 allocations: 7.63 MiB)
 categorical:
  14.033 ms (4 allocations: 688 bytes)
String
 raw:
  38.073 ms (4 allocations: 528 bytes)
 categorical:
  34.223 ms (4 allocations: 688 bytes)
Union{Missing, Int64}
 raw:
  7.391 ms (4 allocations: 560 bytes)
 categorical:
  20.898 ms (1000004 allocations: 30.52 MiB)
Union{Missing, String}
 raw:
  27.825 ms (4 allocations: 528 bytes)
 categorical:
  41.926 ms (1000004 allocations: 30.52 MiB)


### When aggregating use column selector and prefer integer, categorical, or pooled array grouping variable

In [12]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [13]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,c,1
2,c,1
3,c,1
4,c,1
5,c,1
6,c,1
7,c,1
8,c,1
9,c,1


In [14]:
@btime combine(v -> sum(v.y), $gdf) # traditional syntax, slow

  52.125 ms (334 allocations: 19.08 MiB)


,x,x1
,Char,Int64
1,c,2498871
2,b,2500120
3,d,2500686
4,a,2500323


In [15]:
@btime combine($gdf, :y=>sum) # use column selector

  11.406 ms (230 allocations: 13.56 KiB)


,x,y_sum
,Char,Int64
1,c,2498871
2,b,2500120
3,d,2500686
4,a,2500323


In [16]:
transform!(df, :x => categorical => :x);

In [17]:
gdf = groupby(df, :x)

,x,y
,Cat…,Int64
1,a,1
2,a,1
3,a,1
4,a,1
5,a,1
6,a,1
7,a,1
8,a,1
9,a,1


In [18]:
@btime combine($gdf, :y=>sum)

  10.575 ms (237 allocations: 14.14 KiB)


,x,y_sum
,Cat…,Int64
1,a,2500323
2,b,2500120
3,c,2498871
4,d,2500686


In [19]:
transform!(df, :x => PooledArray{Char} => :x)

,x,y
,Char,Int64
1,c,1
2,b,1
3,c,1
4,d,1
5,d,1
6,b,1
7,a,1
8,c,1
9,c,1


In [20]:
gdf = groupby(df, :x)

,x,y
,Char,Int64
1,c,1
2,c,1
3,c,1
4,c,1
5,c,1
6,c,1
7,c,1
8,c,1
9,c,1


In [21]:
@btime combine($gdf, :y=>sum)

  11.303 ms (232 allocations: 13.62 KiB)


,x,y_sum
,Char,Int64
1,c,2498871
2,b,2500120
3,d,2500686
4,a,2500323


### Use views instead of materializing a new DataFrame

In [22]:
x = DataFrame(rand(100, 1000), :auto)

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134
2,0.701979,0.9453,0.37646,0.0947803,0.251327,0.236166,0.293697,0.203283
3,0.506292,0.674179,0.642127,0.92571,0.554715,0.658771,0.321764,0.745311
4,0.45687,0.567372,0.418623,0.581003,0.0698292,0.960586,0.82162,0.623989
5,0.248872,0.619047,0.709611,0.501256,0.416595,0.842667,0.508236,0.120467
6,0.00838456,0.421281,0.666596,0.862949,0.509964,0.0691401,0.597594,0.181125
7,0.925194,0.431258,0.658452,0.149838,0.677729,0.46362,0.307422,0.543076
8,0.657568,0.553293,0.211001,0.702664,0.98197,0.829002,0.322775,0.605153
9,0.0989121,0.850505,0.802189,0.103567,0.0792299,0.929949,0.451498,0.989321


In [23]:
@btime $x[1:1, :]

  208.100 μs (2985 allocations: 159.44 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134


In [24]:
@btime $x[1, :]

  27.593 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134


In [25]:
@btime view($x, 1:1, :)

  27.291 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134


In [26]:
@btime $x[1:1, 1:20]

  5.214 μs (50 allocations: 4.22 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134


In [27]:
@btime $x[1, 1:20]

  30.172 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134


In [28]:
@btime view($x, 1:1, 1:20)

  30.272 ns (0 allocations: 0 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.777016,0.896444,0.225618,0.0398342,0.798797,0.525025,0.871787,0.645134
